# Fixed Effects Model

The purpose of this program is to regress a mortgage approval variable against race, ethnicity, gender, and other control variables found in HMDA data. Using the model below.

$P(Approval = 1 | Race/Sex, \chi_ji, \alpha_i) = \beta_0 + \lambda_ji * Race/Sex + \beta_ji * \chi_ji + \alpha_i + \mu $

Where $\lambda_ji$ are the variables of interest, $\beta_ji$ are the coefficients on the control variables, 
alpha_i are the fixed effects, and $\chi_j$ are the control variables.

Variables of Interest
- White
- Black
- Asian
- Hispanic
- Other
- Male 
- Female

Control Variables
- Income (log)
- Loan to Value ratio
- Debt to Income ratio
- Loan Amount (log)
- Pre-Approval indicators

Variables ommited in model to prevent perfect collinearity.
- Race - White
- Sex - Male

Filters
- Loan Purpose
- Occupancy Type

Clustered Standard errors
- by Lender
- by State
- by County
- by Census Tract

https://timeseriesreasoning.com/contents/the-fixed-effects-regression-model-for-panel-data-sets/

In [1]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS
from linearmodels.panel import compare
import statsmodels.api as sm
import statsmodels.formula.api as smf

# np.set_printoptions(precision=3, suppress=True)

#This will allow all columns to be displayed when reviewing the data.
pd.options.display.max_columns = None

C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [ ]:
'''
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)
import sys
print(sys.version)
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
'''

## Load in and manipulate dataset.

Below is for manipulating the dataset before running it through the funciton.

In [2]:
# Load in HMDA Data
HMDA_clean_file_location = r'HMDA Clean IL SAMPLE.csv'
HMDA_clean_0 = pd.read_csv(HMDA_clean_file_location)
#HMDA_clean_0

### Further Cleaning

In [3]:
#Clean df
HMDA_clean_1 = HMDA_clean_0.copy()
HMDA_clean_1 = HMDA_clean_1.dropna()
HMDA_clean_1['Census_Tract'] = HMDA_clean_1['Census_Tract'].apply(str)

#Filter Occupancy type to Principoal residence. Omits secondary residence purposes and investment purposes.
# "Occupancy_Type" = 1, Second Residence" = 2, "Investment Property" = 3.
HMDA_clean_2_1 = HMDA_clean_1[HMDA_clean_1["Occupancy_Type"] == 1]

#Sets County_Code and Census_Tract as strings.
HMDA_clean_2 = HMDA_clean_2_1.copy()
HMDA_clean_2['County_Code'] = HMDA_clean_2['County_Code'].astype(str)
HMDA_clean_2['Census_Tract'] = HMDA_clean_2['Census_Tract'].astype(str)
#HMDA_clean_2

### Set Index

In [4]:
HMDA_Lender_LEI_index = HMDA_clean_2.set_index(['Lender_LEI', 'Year'])
HMDA_State_index = HMDA_clean_2.set_index(['State','Year'])
HMDA_County_Code_index = HMDA_clean_2.set_index(['County_Code','Year'])
HMDA_Census_Tract_index = HMDA_clean_2.set_index(['Census_Tract','Year'])
HMDA_Lender_LEI_index

index State County_Code   Census_Tract  Approved  \
Lender_LEI           Year                                                       
54930034MNPILHP25H80 2021        2    IL     17163.0  17163503404.0         1   
                     2021       13    IL     17197.0  17197880309.0         1   
FT6J43S06X6CLJ0R0B48 2021      149    IL     17031.0  17031340300.0         1   
                     2021      158    IL     17097.0  17097860906.0         0   
                     2021      167    IL     17197.0  17197880203.0         1   
...                            ...   ...         ...            ...       ...   
549300MXJA09WZJ0DV55 2019  1783143    IL     17031.0  17031828505.0         1   
                     2019  1783146    IL     17111.0  17111871109.0         0   
                     2019  1783147    IL     17197.0  17197884005.0         1   
                     2019  1783148    IL     17073.0  17073030203.0         1   
                     2019  1783150    IL     17073.0  17073030203.0         1   

                           Denied     Race     Sex  Income  Log_Income  \
Lender_LEI           Year                                                
54930034MNPILHP25H80 2021       0  0_White  0_Male    65.0    4.174387   
                     2021       0  0_White  0_Male   151.0    5.017280   
FT6J43S06X6CLJ0R0B48 2021       0    Asian  0_Male    76.0    4.330733   
                     2021       1    Asian  0_Male    48.0    3.871201   
                     2021       0    Asian  0_Male    44.0    3.784190   
...                           ...      ...     ...     ...         ...   
549300MXJA09WZJ0DV55 2019       0    Black  Female    47.0    3.850148   
                     2019       1   Latinx  Female    42.0    3.737670   
                     2019       0  0_White  Female    50.0    3.912023   
                     2019       0  0_White  Female    94.0    4.543295   
                     2019       0  0_White  Female    55.0    4.007333   

                           Loan_Amount  Log_Loan_Amount     LTV     Loan_Type  \
Lender_LEI           Year                                                       
54930034MNPILHP25H80 2021     235000.0        12.367341  76.000  Conventional   
                     2021     665000.0        13.407542  98.230            VA   
FT6J43S06X6CLJ0R0B48 2021     245000.0        12.409013  68.572  Conventional   
                     2021     165000.0        12.013701  80.000  Conventional   
                     2021     145000.0        11.884489  64.318  Conventional   
...                                ...              ...     ...           ...   
549300MXJA09WZJ0DV55 2019     135000.0        11.813030  96.500           FHA   
                     2019     155000.0        11.951180  62.240           FHA   
                     2019     155000.0        11.951180  90.000  Conventional   
                     2019     165000.0        12.013701  97.000  Conventional   
                     2019     125000.0        11.736069  80.000  Conventional   

                          DTI_Ratio               Preapproval  Occupancy_Type  
Lender_LEI           Year                                                      
54930034MNPILHP25H80 2021  30%-<36%  0 No Preapproval Request               1  
                     2021        41  0 No Preapproval Request               1  
FT6J43S06X6CLJ0R0B48 2021  30%-<36%  0 No Preapproval Request               1  
                     2021      >60%  0 No Preapproval Request               1  
                     2021  30%-<36%  0 No Preapproval Request               1  
...                             ...                       ...             ...  
549300MXJA09WZJ0DV55 2019        36  0 No Preapproval Request               1  
                     2019        46  0 No Preapproval Request               1  
                     2019  30%-<36%  0 No Preapproval Request               1  
                     2019  30%-<36%  0 No Preapproval Request               1 

## Run Models

OLS - Race*Sex Only

In [5]:
OLS_Race_Sex_Indicators = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex', data = HMDA_Lender_LEI_index).fit(cov_type = "clustered", cluster_entity = True)
#OLS_Race_Sex_Indicators

FE_Model_0 - Race*Sex Only

In [6]:
OLS_Model_1 = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex + Log_Income + Log_Loan_Amount + LTV + DTI_Ratio', data = HMDA_Lender_LEI_index).fit(cov_type = "clustered", cluster_entity = True)
#OLS_Model_1

FE_Model_1 includes Log Income, Log Loan Amount, LTV, and DTI

In [7]:
FE_Model_1 = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex + Log_Income + Log_Loan_Amount + LTV*DTI_Ratio + EntityEffects + TimeEffects', data = HMDA_Lender_LEI_index).fit(cov_type = "clustered", cluster_entity = True)
#FE_Model_1

FE_Model_2 adds preapproval and loan type.

In [8]:
FE_Model_2 = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex + Log_Income + Log_Loan_Amount + LTV*DTI_Ratio + Preapproval + Loan_Type + EntityEffects + TimeEffects', data = HMDA_Lender_LEI_index).fit(cov_type = "clustered", cluster_entity = True)
#FE_Model_2

FE_Model_3 changes the index to State. Adds Lender Indicators.

In [9]:
#Will only work with more than one state. The current test sample is for IL only.

#FE_Model_3 = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex + Log_Income + Log_Loan_Amount + LTV*DTI_Ratio + Preapproval + Loan_Type + Lender_LEI + EntityEffects', data = HMDA_State_index).fit(cov_type = "clustered", cluster_entity = True)
#FE_Model_3

FE_Model_4 changes the index to County Code.

In [19]:
FE_Model_4 = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex + Log_Income + Log_Loan_Amount + LTV*DTI_Ratio + Preapproval + Loan_Type + Lender_LEI + EntityEffects + TimeEffects', data = HMDA_County_Code_index).fit(cov_type = "clustered", cluster_entity = True)
#FE_Model_4

FE_Model_5 changes the index to Census Tract.

In [11]:
#This current model may not work if there are less than 2 cases for each census tract. Drop census tracts that may have this??? 

#FE_Model_5 = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex + Log_Income + Log_Loan_Amount + LTV*DTI_Ratio + Preapproval + Loan_Type + Lender_LEI + EntityEffects', data = HMDA_Census_Tract_index).fit(cov_type = "clustered", cluster_entity = True)
#FE_Model_5

FE_Model_6 changes the covariance type to unadjusted.

In [21]:
FE_Model_6 = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex + Log_Income + Log_Loan_Amount + LTV*DTI_Ratio + Preapproval + Loan_Type + Lender_LEI + EntityEffects + TimeEffects', data = HMDA_County_Code_index).fit()
#FE_Model_6

FE_Model_7 changes the covariance type to unadjusted, adds County Indicators, and changes index to lender.

In [20]:
FE_Model_7 = PanelOLS.from_formula('Approved ~ 1 + (Race)*Sex + Log_Income + Log_Loan_Amount + LTV*DTI_Ratio + Preapproval + Loan_Type + County_Code + EntityEffects + TimeEffects', data = HMDA_Lender_LEI_index).fit()
#FE_Model_7

# Compare Models

A link to the documentation to build a table from. https://bashtage.github.io/linearmodels/panel/panel/linearmodels.panel.results.PanelEffectsResults.html

In [29]:
comparison_2 = compare({'OLS - Model 0' : OLS_Race_Sex_Indicators,
         'OLS - Model 1' : OLS_Model_1,
         'FE - Model 1' : FE_Model_1,
         'FE - Model 2' : FE_Model_2,
         'FE - Model 4' : FE_Model_4,
         'FE - Model 6' : FE_Model_6,
         'FE - Model 7' : FE_Model_7 
        }, stars = True).params

#Pull rows from model comparisons.
Model_Comparison_1 = comparison_2.loc[['Race[T.Asian]', 'Race[T.Black]',
                                   'Race[T.Other]','Race[T.Latinx]', 'Sex[T.Female]']]
Model_Comparison_2 = Model_Comparison_1.set_axis(['Asian', 'Black', 'Other', 'LatinX', 'Female'])


#Add rows that give information on each regression.
Model_Comparison_2.loc['LTV + DTI Interactions'] = ['No', 'Yes', 'Yes', 'Yes','Yes','Yes','Yes']
Model_Comparison_2.loc['Preapproval + Loan Type'] = ['No', 'No', 'No', 'Yes','Yes','Yes','Yes']
Model_Comparison_2.loc['Lender Indicators'] = ['No', 'No', 'No', 'No','Yes','Yes','No']
Model_Comparison_2.loc['County Code Indicator'] = ['No', 'No', 'No', 'No','No','No','Yes']
Model_Comparison_2.loc['Entity/Time Effects'] = ['No', 'no', 'Yes', 'Yes','Yes','Yes','Yes']
Model_Comparison_2.loc['Index'] = ['Lender', 'Lender', 'Lender', 'Lender','County','County','Lender']
Model_Comparison_2.loc['Cov. Estimator'] = ['Clustered', 'Clustered', 'Clustered', 'Clustered','Clustered','Normal','Normal']
Model_Comparison_2.loc['Overal R Squared'] = [OLS_Race_Sex_Indicators.rsquared_overall, OLS_Model_1.rsquared_overall, 
                                                FE_Model_1.rsquared_overall, FE_Model_2.rsquared_overall, FE_Model_4.rsquared_overall,
                                                FE_Model_6.rsquared_overall, FE_Model_7.rsquared_overall]

Model_Comparison_2

,OLS - Model 0,OLS - Model 1,FE - Model 1,FE - Model 2,FE - Model 4,FE - Model 6,FE - Model 7
Asian,-0.016828,-0.019593,-0.014709,-0.013703,-0.01177,-0.01177,-0.01177
Black,-0.108117,-0.070996,-0.062679,-0.063096,-0.059224,-0.059224,-0.059224
Other,-0.071853,-0.047668,-0.039594,-0.039446,-0.039261,-0.039261,-0.039261
LatinX,-0.054195,-0.030736,-0.022152,-0.021498,-0.01844,-0.01844,-0.01844
Female,-0.001687,0.00434,0.001703,0.002427,0.002612,0.002612,0.002612
LTV + DTI Interactions,No,Yes,Yes,Yes,Yes,Yes,Yes
Preapproval + Loan Type,No,No,No,Yes,Yes,Yes,Yes
Lender Indicators,No,No,No,No,Yes,Yes,No
County Code Indicator,No,No,No,No,No,No,Yes
Entity/Time Effects,No,no,Yes,Yes,Yes,Yes,Yes


### For Distributed Computing

Idea behind distribution
https://stats.stackexchange.com/questions/263429/how-to-run-linear-regression-in-a-parallel-distributed-way-for-big-data-setting

SGD vs OLS
https://www.youtube.com/watch?v=fkS3FkVAPWU

Example of Fixed Effects
https://www.youtube.com/watch?v=FCm3_Id6RKM&t=532s

Fixed Effects Model Explanation
https://www.youtube.com/watch?v=J9UEYUXi6lY&t=28s

TensorFlow Examples(OLS and SGD)
https://github.com/jldbc/Tensorflow_ML_Algorithms

OLS From Scratch
https://jianghaochu.github.io/ordinary-least-squares-regression-in-python-from-scratch.html, 
https://www.youtube.com/watch?v=KYNuzfn5Fx0